<a href="https://colab.research.google.com/github/aiscience-22/UA_War/blob/machine_learning_0.01/Postgres_Reading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://jessehernandez70test1.s3.us-west-1.amazonaws.com/joined_v001.csv

In [ ]:
# The complete commands to create the table in database and populate the data

!curl -s -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/postgresql/AirQualityUCI.sql

!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d $TFIO_DEMO_DATABASE_NAME -f AirQualityUCI.sql

# Create Dataset from PostgreSQL server and use it in TensorFlow

Create a Dataset from PostgreSQL server is as easy as calling tfio.experimental.IODataset.from_sql with query and endpoint arguments. The query is the SQL query for select columns in tables and the endpoint argument is the address and database name:

In [ ]:
import os
import tensorflow_io as tfio

endpoint="postgresql://{}:{}@{}?port={}&dbname={}".format(
    os.environ['TFIO_DEMO_DATABASE_USER'],
    os.environ['TFIO_DEMO_DATABASE_PASS'],
    os.environ['TFIO_DEMO_DATABASE_HOST'],
    os.environ['TFIO_DEMO_DATABASE_PORT'],
    os.environ['TFIO_DEMO_DATABASE_NAME'],
)

dataset = tfio.experimental.IODataset.from_sql(
    query="SELECT co, pt08s1 FROM AirQualityUCI;",
    endpoint=endpoint)

print(dataset.element_spec)

The output of dataset.element_spec above, the element of the created Dataset is a python dict object with column names of the database table as keys. It is quite convenient to apply further operations. For example, you could select both nox and no2 field of the Dataset, and calculate the difference:

In [ ]:
dataset = tfio.experimental.IODataset.from_sql(
    query="SELECT nox, no2 FROM AirQualityUCI;",
    endpoint=endpoint)

dataset = dataset.map(lambda e: (e['nox'] - e['no2']))

# check only the first 20 record
dataset = dataset.take(20)

print("NOx - NO2:")
for difference in dataset:
  print(difference.numpy())